In [40]:
import jieba
import jieba.analyse
import re
import pandas as pd
import urllib.parse as ur
import os 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
#os.getcwd()
df = pd.DataFrame(pd.read_csv('news.csv')) #讀取新聞文章並以變數df儲存
print(df.columns) 

Index(['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author',
       'content', 'page_url'],
      dtype='object')


In [41]:
#--------------------------------
# 存停用詞, 分詞, 過濾後分詞的list
#--------------------------------
stopWords=[]
#--------------------------------
# 讀入停用詞檔
#--------------------------------
with open('stopword.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopWords.append(data)

In [42]:
#找包含關鍵字文章
mtk_up = df["content"].str.contains('聯發科')  #為一布林值list，儲存(df)新聞內文(content欄位)中是否含有「聯發科」
#b= df[df["content"].isin['聯發科',"上漲"]]
count_down = 0 #計算總共有幾篇文章被標記為跌
index_down = [] #儲存被標記為跌的文章在df中的index
count_up = 0 #計算總共有幾篇文章被標記為漲
index_up = [] #儲存被標記為漲的文章在df中的index
for i in range(len(mtk_up)):
    # 文章包含「上漲」或「漲」字詞但不包含「下跌」或「跌」字詞才可以被標記為漲
    # 文章包含「下跌」或「跌」字詞但不包含「上漲」或「漲」字詞才可以被標記為跌
    if (mtk_up[i] == True) and((df['content'][i].find('下跌')!=-1) or (df['content'][i].find('跌')!=-1))and ((df['content'][i].find('上漲')==-1) and (df['content'][i].find('漲')==-1)):
         count_down += 1 
         index_down += [i]
    elif (mtk_up[i] == True) and((df['content'][i].find('上漲')!=-1) or (df['content'][i].find('漲')!=-1))and((df['content'][i].find('下跌')==-1) and (df['content'][i].find('跌')==-1)):
         count_up += 1
         index_up += [i]
print(count_down) 
print(count_up)

298
603


In [43]:
text_all = [] #存取jieba切詞後的所有新聞內容，長相會是["財經 新聞 內容 報導......", "聯發科 股票 本月 預估 ......",......]
label_all = [] #存取各篇新聞的漲跌標記

sent_down = [] #存取jieba切詞後的所有下跌新聞內容，長相會是["財經 新聞 內容 報導......", "聯發科 股票 本月 預估 ......",......]
for i in index_down: 
    text_down = df.iloc[i].content #提取下跌文章的內文並暫時以text_down變數儲存
    text_down = re.sub(r'[^\w]','',text_down) #將無意義的符號刪掉
    text_down = re.sub(r'[A-Za-z0-9]','',text_down) #將數字或是英文刪掉
    temp_down = '' #暫時儲存以空白隔開的字詞string
    segments_down=[] #儲存所有jieba切割出的字詞
    remainderWords_down=[] #儲存刪除stopWord之後的字詞list
    segments_down = jieba.cut(text_down, cut_all=False) #以jieba切割text_down字串並將所有切割出的字詞儲存至segments_down
    remainderWords_down = list(filter(lambda a: a not in stopWords and a != '\n', segments_down)) #刪除segments_down中是stopWords字詞
    
    cnt = 0 
    for k in remainderWords_down: #這個迴圈的用意是要將remainderWords_down中的字詞list以一條string形式儲存，每個字詞以空白隔開，之後才能讀入KNN套件
        if (cnt == 0) or (cnt == len(remainderWords_down)): 
            temp_down += (k)
        else:
            temp_down += ' '
            temp_down += k
        cnt += 1
    sent_down.append(temp_down) #將temp_down append到sent_down中
    
sent_up = [] #存取jieba切詞後的所有上漲新聞內容，長相會是["財經 新聞 內容 報導......", "聯發科 股票 本月 預估 ......",......]
for i in index_up:
    text_up = df.iloc[i].content
    text_up = re.sub(r'[^\w]','',text_up)
    text_up = re.sub(r'[A-Za-z0-9]','',text_up)
    temp_up = ''
    segments_up=[]
    remainderWords_up=[]
    segments_up = jieba.cut(text_up, cut_all=False)
    remainderWords_up = list(filter(lambda a: a not in stopWords and a != '\n', segments_up))
    cnt = 0
    for k in remainderWords_up:
        if (cnt == 0) or (cnt == len(remainderWords_up)):
            temp_up += (k)
        else:
            temp_up += ' '
            temp_up += k
        cnt += 1
    sent_up.append(temp_up)

for i in range(len(sent_down)):
    text_all.append(sent_down[i]) #將sent_down所有以字詞組成的文章存到text_all中
    label_all.append(int(0)) #將label_all中同一index的儲存格存取"0"，代表下跌
    
for i in range(len(sent_up)):
    text_all.append(sent_up[i]) #將sent_up所有以字詞組成的文章存到text_all中
    label_all.append(int(1)) #將label_all中同一index的儲存格存取"1"，代表上漲
    
#label_all的長相會是前面全都是0後面全都是1，[0,0,0,0.....,1,1,1,]，剛好配合下跌與上漲文章在text_all中的順序，這樣讀入KNN套件時才能成功

In [44]:
from sklearn.feature_extraction.text import CountVectorizer #利用套件抓取所有文章中的關鍵字
from sklearn.model_selection import train_test_split 

cv = CountVectorizer()
cv_fit=cv.fit_transform(text_all)
print(cv_fit.toarray()) #這個是每個文章的對上所有關鍵字tf list
print(cv_fit.toarray().shape) #(文章數, 關鍵字數量)
print(cv_fit) #(文章對上所有關鍵字的tf細項 ex:(第X篇文章 , 第Y個關鍵字) tf)
# print(cv.get_feature_names())
x = cv_fit 
y = label_all #將所有文章的漲跌標記以y變數儲存

#利用train_test_split將所有文章隨意分割成training set和testing set
#X_train, X_test, y_train, y_test = 文章的training data,文章的testing data, 漲跌標記的training data,漲跌標記的testing dat 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101) 

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(901, 40661)
  (0, 40269)	2
  (0, 33242)	3
  (0, 32448)	1
  (0, 19062)	1
  (0, 25468)	1
  (0, 10750)	1
  (0, 26605)	3
  (0, 24895)	6
  (0, 26173)	2
  (0, 9827)	1
  (0, 32201)	2
  (0, 24876)	2
  (0, 34422)	1
  (0, 18496)	2
  (0, 35222)	2
  (0, 36373)	1
  (0, 10751)	1
  (0, 2842)	1
  (0, 37114)	1
  (0, 32697)	1
  (0, 2247)	1
  (0, 24160)	1
  (0, 15463)	6
  (0, 6472)	2
  (0, 20638)	1
  :	:
  (900, 11979)	1
  (900, 3996)	1
  (900, 39014)	1
  (900, 10253)	1
  (900, 12511)	1
  (900, 35402)	1
  (900, 40626)	1
  (900, 28769)	1
  (900, 24082)	1
  (900, 32937)	1
  (900, 540)	1
  (900, 1200)	1
  (900, 14533)	1
  (900, 1277)	1
  (900, 8293)	1
  (900, 19202)	1
  (900, 28741)	1
  (900, 10053)	1
  (900, 13298)	1
  (900, 31366)	1
  (900, 13438)	1
  (900, 32271)	1
  (900, 16574)	1
  (900, 6294)	1
  (900, 35906)	1


In [62]:
from sklearn.neighbors import KNeighborsClassifier #KNN套件
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#嘗試每次以不同鄰居數量決定testing文章的漲跌標記
accuracy = []
TN = []
FP = []
FN = []
TP = []

k_cnt = [1,3,5,7,9,11,13, 15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59] #KNN存取幾個鄰居的數量
for i in k_cnt:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train) #將training set給KNN套件建出模型
    pred = knn.predict(X_test) #將文章的testing set讀入模型中並決定文章的漲跌標記
#     %matplotlib inline
#     class_names=[0,1] # name of classes
#     fig, ax = plt.subplots()
#     tick_marks = np.arange(len(class_names))
#     plt.xticks(tick_marks, class_names)
#     plt.yticks(tick_marks, class_names)
#     # create heatmap
#     matrix = confusion_matrix(y_test,pred)
#     sns.heatmap(pd.DataFrame(matrix), annot=True, cmap="YlGnBu" ,fmt='g')
#     ax.xaxis.set_label_position("top")
#     plt.tight_layout()
#     plt.title('Confusion matrix', y=1.1)
#     plt.ylabel('Actual label')
#     plt.xlabel('Predicted label')
    print('k=', i)
#     print(pred)
    print(confusion_matrix(y_test,pred))
    print("Accuracy=", accuracy_score(y_test, pred))
    print('R=',metrics.recall_score(y_test, pred))
    print('P=',metrics.precision_score(y_test, pred))
    print('F1=',metrics.f1_score(y_test, pred))
    print("---------------------------------------------------------------------------------")
    accuracy.append(accuracy_score(y_test, pred))
    TN.append(confusion_matrix(y_test,pred)[0][0])
    FP.append(confusion_matrix(y_test,pred)[0][1])
    FN.append(confusion_matrix(y_test,pred)[1][0])
    TP.append(confusion_matrix(y_test,pred)[1][1])

k= 1
[[ 32  34]
 [ 11 104]]
Accuracy= 0.7513812154696132
R= 0.9043478260869565
P= 0.7536231884057971
F1= 0.8221343873517787
---------------------------------------------------------------------------------
k= 3
[[ 27  39]
 [ 10 105]]
Accuracy= 0.7292817679558011
R= 0.9130434782608695
P= 0.7291666666666666
F1= 0.8108108108108107
---------------------------------------------------------------------------------
k= 5
[[ 25  41]
 [  5 110]]
Accuracy= 0.7458563535911602
R= 0.9565217391304348
P= 0.7284768211920529
F1= 0.8270676691729324
---------------------------------------------------------------------------------
k= 7
[[ 20  46]
 [  5 110]]
Accuracy= 0.7182320441988951
R= 0.9565217391304348
P= 0.7051282051282052
F1= 0.8118081180811809
---------------------------------------------------------------------------------
k= 9
[[ 22  44]
 [  5 110]]
Accuracy= 0.7292817679558011
R= 0.9565217391304348
P= 0.7142857142857143
F1= 0.8178438661710038
----------------------------------------------------

In [57]:
Output_result = {"K":k_cnt, "accuracy":accuracy, "TN":TN, "FP":FP, "FN":FN, "TP":TP}
Output = pd.DataFrame(Output_result)
Output.to_csv("KNNresult.csv", encoding = "utf_8_sig")